In [47]:
!pip install -q pyinterval
from interval import interval
from interval import imath
import pandas as pd
import numpy as np
import math
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_midpoint(interv):
    return (interv[0].sup + interv[0].inf) / 2

def interval_to_str(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

In [48]:
def hansen(f, df, start, end, e, tbl):
    x = interval([start, end])
    width = calculate_width(x)
    Fx = f(x)

    # If no 0 in F(X) -> no root
    if 0 not in Fx:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
        return

    # If interval width less than epsilon -> root
    if width < e:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, 'Root']
        return
        
    # Half if 0 in F'(X)
    x_middle = calculate_midpoint(x)
    f_middle = f(x_middle)
    dFx = df(x)
    if f_middle == 0.0:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, 'Root?']
        hansen(f, df, start, x_middle, tbl)
        hansen(f, df, x_middle, end, tbl)
        return

    # Stop if x_i+1 is empty
    U = x_middle - f_middle / dFx
    x_next = U & x
    if not x_next:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
        return

    # Continue with narrowed intervals
    tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
    [hansen(f, df, x.inf, x.sup, e, tbl) for x in x_next]

In [49]:
func = lambda x: imath.sin(x-1)*(x-2)*(x-3)
dfunc = lambda x: (x**2-5*x+6)*imath.cos(x-1)-(5-2*x)*imath.sin(x-1)
e = 1e-6
a = 0.9
b = 2.3

In [50]:
hansen_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Root'])
hansen(func, dfunc, a, b, e, hansen_result)
hansen_result

,Interval,Interval extension,Width,Root
0,"[0.9000000, 2.3000000]","[-0.6070417, 2.2258194]",1.400000e+00,
1,"[0.9000000, 1.5555241]","[-0.2306152, 1.2182678]",6.555241e-01,
2,"[0.9000000, 1.1548638]","[-0.2306152, 0.3563072]",2.548638e-01,
3,"[0.9280922, 1.0107439]","[-0.1595619, 0.0238605]",8.265163e-02,
4,"[0.9939627, 1.0055862]","[-0.0121840, 0.0112737]",1.162347e-02,
5,"[0.9999936, 1.0000067]","[-0.0000129, 0.0000133]",1.312370e-05,
6,"[1.0000000, 1.0000000]","[-0.0000000, 0.0000000]",7.381540e-12,Root
7,"[1.3441439, 1.5555241]","[0.2166167, 0.5727458]",2.113802e-01,
8,"[1.6406772, 2.3000000]","[-0.3929360, 0.4706361]",6.593228e-01,
9,"[1.6406772, 1.9590661]","[0.0254694, 0.3998606]",3.183888e-01,
